### 0. 환경설정

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

### 1. 버스
* [후과정]공식명칭 행정동 추출_(동만 추출하는 과정은 X)
* 전체과정은 '2.자전거'로 가기

In [4]:
df = pd.read_csv("data/행정구역전처리.csv", index_col=0)
df.head()

,자치구명,행정구역명,법정동,행정구역분류
2,종로구,청운효자동,청운동,1101072
3,종로구,청운효자동,신교동,1101072
4,종로구,청운효자동,궁정동,1101072
5,종로구,청운효자동,효자동,1101072
6,종로구,청운효자동,창성동,1101072


In [5]:
df['행정구역명'].unique()

array(['청운효자동', '사직동', '삼청동', '부암동', '평창동', '무악동', '교남동', '가회동',
       '종로1.2.3.4가동', '종로5.6가동', '이화동', '혜화동', '창신1동', '창신2동', '창신3동',
       '숭인1동', '숭인2동', '소공동', '회현동', '명동', '필동', '장충동', '광희동', '을지로동',
       '신당동', '다산동', '약수동', '청구동', '신당5동', '동화동', '황학동', '중림동', '후암동',
       '용산2가동', '남영동', '청파동', '원효로1동', '원효로2동', '효창동', '용문동', '한강로동',
       '이촌1동', '이촌2동', '이태원1동', '이태원2동', '한남동', '서빙고동', '보광동', '왕십리2동',
       '왕십리도선동', '마장동', '사근동', '행당1동', '행당2동', '응봉동', '금호1가동', '금호2.3가동',
       '금호4가동', '옥수동', '성수1가1동', '성수1가2동', '성수2가1동', '성수2가3동', '송정동',
       '용답동', '화양동', '군자동', '중곡1동', '중곡2동', '중곡3동', '중곡4동', '능동', '광장동',
       '자양1동', '자양2동', '자양3동', '자양4동', '구의1동', '구의2동', '구의3동', '용신동',
       '제기동', '전농1동', '전농2동', '답십리1동', '답십리2동', '장안1동', '장안2동', '청량리동',
       '회기동', '휘경1동', '휘경2동', '이문1동', '이문2동', '면목2동', '면목4동', '면목5동',
       '면목본동', '면목7동', '면목3.8동', '상봉1동', '상봉2동', '중화1동', '중화2동', '묵1동',
       '묵2동', '망우본동', '망우3동', '신내1동', '신내2동', '성북동', '삼선동', '동선동', '돈암1동',
    

In [35]:
df2 = pd.read_csv("dong_bus.csv", index_col=0,encoding = 'cp949')
df2['0'] = df2['0'].str.replace(' ','')
df2['0'] = df2['0'].str.replace('·','.')
df2.head()

,0,1
1,성북동,True
2,종로1.2.3.4가동,True
3,혜화동,True
5,성북동,True
7,혜화동,True


In [36]:
df2.shape

(11531, 2)

In [30]:
"1동" in df['행정구역명'].unique()

False

In [37]:
df2["final"] = df2['0'].isin(df['행정구역명'].unique())
df2.head()

,0,1,final
1,성북동,True,True
2,종로1.2.3.4가동,True,True
3,혜화동,True,True
5,성북동,True,True
7,혜화동,True,True


In [39]:
df2 = df2[df2['final']==True]
df2.shape

(9707, 3)

In [40]:
df2.head()

,0,1,final
1,성북동,True,True
2,종로1.2.3.4가동,True,True
3,혜화동,True,True
5,성북동,True,True
7,혜화동,True,True


In [43]:
df2 = df2.drop(['1','final'],axis=1)
df2.head()

,0
1,성북동
2,종로1.2.3.4가동
3,혜화동
5,성북동
7,혜화동


In [44]:
df2.to_csv('bus_dong_final.csv')

### 2. 자전거
* [전과정]원본 전처리->공식명칭 행정동 추출

In [46]:
df = pd.read_csv("data/자전거주소변환.csv", index_col=0)
df.head(3)

,대여소번호,대여소명,자치구,상세주소,위도,경도,설치시기,설치형태LCD,설치형태QR,운영방식,address
4,102.0,망원역 1번출구 앞,마포구,서울특별시 마포구 월드컵로 72,37.555649,126.910629,2022-05-03,NaN,15.0,QR,"스타벅스, 74, 월드컵로, 서교동, 마포구, 서울, 04004, 대한민국"
5,103.0,망원역 2번출구 앞,마포구,서울특별시 마포구 월드컵로 79,37.554951,126.910835,2022-05-03,NaN,14.0,QR,"기업은행서교동지점 (최규하대통령가옥), 월드컵로11길, 19통, 망원1동, 마포구,..."
6,104.0,합정역 1번출구 앞,마포구,서울특별시 마포구 양화로 59,37.550629,126.914986,2022-05-12,NaN,13.0,QR,"합정역 1번출구, 양화로, 홍대거리, 서교동, 마포구, 서울, 04045, 대한민국"


In [48]:
df = df[df['address'].str.contains('서울')] ## df[조건식]
df.shape

(2645, 11)

In [50]:
address_list = df['address'].values.tolist()
address_list = ",".join(address_list)
address_list = address_list.split(',')
matching = [i for i in address_list if "동" in i]
matching = pd.DataFrame(zip(matching))
matching.head()

,0
0,서교동
1,기업은행서교동지점 (최규하대통령가옥)
2,망원1동
3,서교동
4,동양한강트레벨 오피스텔


In [55]:
matching[1] = matching[0].str.endswith('동')
matching_final =matching[matching[1]==True]
matching_final[0] = matching_final[0].str.replace(' ','')
matching_final[0] = matching_final[0].str.replace('·','.')
matching_final.head()
#matching.head()

,0,1
0,서교동,True
2,망원1동,True
3,서교동,True
5,서교동,True
6,합정동,True


In [56]:
matching_final.shape

(2782, 2)

In [57]:
df_dong = pd.read_csv("data/행정구역전처리.csv", index_col=0)
matching_final["final"] = matching_final[0].isin(df_dong['행정구역명'].unique())
matching_final = matching_final[matching_final['final']==True]

matching_final.head()

,0,1,final
0,서교동,True,True
2,망원1동,True,True
3,서교동,True,True
5,서교동,True,True
6,합정동,True,True


In [58]:
matching_final.shape

(2332, 3)

In [59]:
matching_final = matching_final.drop([1,'final'],axis = 1)
matching_final.head()

,0
0,서교동
2,망원1동
3,서교동
5,서교동
6,합정동


In [ ]:
matching_final.to_csv('bike_dong_final.csv')

## [221026]지하철_자치구_동

### 1-1. 동

In [33]:
df = pd.read_csv("data/지하철역주소변환.csv", index_col=0,encoding = 'cp949')
df = df[df['address'].str.contains('서울')] ## df[조건식]
print(df.head())
print(df.shape)

    호선  역사명    역사ID         위도          경도  \
1  1호선   남영  1002.0  37.541021  126.971300   
2  1호선   용산  1003.0  37.529849  126.964561   
3  1호선  노량진  1004.0  37.514219  126.942454   
4  1호선   대방  1005.0  37.513342  126.926382   
5  1호선  영등포  1006.0  37.515504  126.907628   

                                     address  
1      2, 한강대로77길, 남영동, 용산구, 서울, 04314, 대한민국  
2  55, 한강대로23길, 한강로동, 한강로3가, 서울, 04377, 대한민국  
3       3, 노량진로, 노량진1동, 동작구, 서울, 06922, 대한민국  
4       2, 대방지하차도, 대방동, 동작구, 서울, 07354, 대한민국  
5    6, 영신로19길, 영등포본동, 영등포구, 서울, 07313, 대한민국  
(390, 6)


In [6]:
address_list = df['address'].values.tolist()
address_list = ",".join(address_list)
address_list = address_list.split(',')

In [7]:
df_dong = pd.read_csv("data/행정구역전처리.csv", index_col=0)
df_dong.head()

,자치구명,행정구역명,행정동,법정동,행정구역분류
2,종로구,청운효자동,청운효자동,청운동,1101072
3,종로구,청운효자동,청운효자동,신교동,1101072
4,종로구,청운효자동,청운효자동,궁정동,1101072
5,종로구,청운효자동,청운효자동,효자동,1101072
6,종로구,청운효자동,청운효자동,창성동,1101072


In [32]:
matching = [i for i in address_list if "동" in i]
matching = pd.DataFrame(zip(matching))
matching[1] = matching[0].str.endswith('동')
matching[0] = matching[0].str.replace(' ','')
matching[0] = matching[0].str.replace('·','.')
matching_final =matching[matching[1]==True]#동으로 끝나기
matching_final["final"] = matching_final[0].isin(df_dong['행정구역명'].unique())
matching_final = matching_final[matching_final['final']==True]#400->343
print(matching_final.head())
print(matching_final.shape)

       0     1  final
0    남영동  True   True
1   한강로동  True   True
2  노량진1동  True   True
4    대방동  True   True
6  영등포본동  True   True
(343, 3)


### 1-2.구


In [28]:
matching2 = [i for i in address_list if "구" in i]
matching2 = pd.DataFrame(zip(matching2))
matching2[1] = matching2[0].str.endswith('구')
matching2[0] = matching2[0].str.replace(' ','')
#matching2[0] = matching2[0].str.replace('·','.')
matching2_final = matching2[matching2[1]==True]#구로 끝나기
matching2_final["final"] = matching2_final[0].isin(df_dong['자치구명'].unique())
matching2_final = matching2_final[matching2_final['final']==True]#471->388
print(matching2_final.head())
print(matching2_final.shape)

      0     1  final
0   용산구  True   True
1   동작구  True   True
2   동작구  True   True
3  영등포구  True   True
4   구로구  True   True
(388, 3)


In [20]:
matching2.head()

,0,1
0,용산구,True
1,동작구,True
2,동작구,True
3,영등포구,True
4,구로구,True


In [ ]:
##

### 다정코드

In [ ]:
#인덱스 재설정
df_mh = df_mh.reset_index(drop=True, inplace=False)

#위도,경도값이 존재하지 않았던 관측치 1개 삭제
df_mh = df_mh[-pd.Series(df_mh['위도'].isnull())]
df_mh


# 가입 없이 좌표->주소 변환->법정동으로 추출되므로 일단 밑으로 내리기
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

import math
import time
import datetime

start = time.time()
address=[]
for i in range(0,(df_mh.shape[0])):
    try:
        address.append(geocoding_reverse(str(df_mh['위도'][i])+','+str(df_mh['경도'][i])))
    except Exception as e:
        print(e)
end = time.time()
sec = (end - start)
result = datetime.timedelta(seconds=sec)
print(result)
result_list = str(datetime.timedelta(seconds=sec)).split(".")
print(result_list[0])


a = []
for i in range(0, len(address)):
    a.append(len(str(address[i]).split(',')))
pd.DataFrame(a).value_counts()


b = pd.DataFrame({'a':a, '행정동명':address, '찐행정동명':[str(0) for x in range(0,len(b))]})

for i in range(0, b.shape[0]):
    if b.iloc[i,0]==4:
        b.iloc[i,2] = str(b.iloc[i,1]).split(',')[0]
    elif b.iloc[i,0]==5:
        b.iloc[i,2] = str(b.iloc[i,1]).split(',')[0]
    elif b.iloc[i,0]==6:
        b.iloc[i,2] = str(b.iloc[i,1]).split(',')[1]
    elif b.iloc[i,0]==7:
        b.iloc[i,2] = str(b.iloc[i,1]).split(',')[2]
    elif b.iloc[i,0]==8:
        b.iloc[i,2] = str(b.iloc[i,1]).split(',')[3]
    elif b.iloc[i,0]==9:
        b.iloc[i,2] = str(b.iloc[i,1]).split(',')[4]

b